In [ ]:
# Possible sources:
"""
https://github.com/topics/vtuber
1. https://github.com/1996scarlet/OpenVtuber
2. https://github.com/1996scarlet/faster-mobile-retinaface 
   (faster version of 1)
3. https://github.com/pkhungurn/talking-head-anime-demo 
   (may be slow and bg needs to be removed)
"""